In [ ]:
# 12/29(월) 16:30

# RAG 평가 개요
- RAG 평가: RAG 시스템이 주어진 입력에 대해 얼마나 효과적으로 관련 정보를 검색하고, 이를 기반으로 정확하고 유의미한 응답을 생성하는지를 측정하는 과정. 
- **평가 요소**
    - **검색 단계 평가**
        - 입력 질문에 대해 검색된 문서나 정보의 관련성과 정확성을 평가.
    - **생성 단계 평가**
        - 검색된 정보를 기반으로 생성된 응답의 품질, 정확성등을 평가.
- **평가 방법**
    - 온/오프라인 평가
        1. **오프라인 평가**
            - 미리 준비된 데이터셋을 활용하여 RAG 시스템의 성능을 측정.
        2. **온라인 평가**
            - 실제 사용자 트래픽과 피드백을 기반으로 시스템의 실시간 성능을 평가.
    - 정량적/정성적 평가
        1. 정량적 평가
            - 자동화된 지표를 사용하여 생성된 텍스트의 품질을 평가.
        2. 정성적 평가
            - 전문가나 일반 사용자가 직접 생성된 응답의 품질을 평가하여 주관적인 지표를 평가.

# [RAGAS](https://www.ragas.io/)
- : RAGAS는 RAG 파이프라인을 **정량적 으로 평가하는** 오픈소스 프레임 워크. 
- RAGAS 문서: https://docs.ragas.io/en/stable/
## 설치
- `pip install ragas rapidfuzz`

In [ ]:
# !uv pip install ragas rapidfuzz

## RAGAS 평가 지표 개요
![ragas_score](figures/ragas_score.png)
- **Generation**
    - llm 모델이 생성한 답변에 대한 평가 지표들.
    - **Faithfulness(신뢰성)**
        -  생성된 답변과 검색된 문서(context)간의 관련성을 평가하는 지표
        -  생성된 답변이 주어진 문맥(context)에 얼마나 충실한지를 평가하는 지표로 할루시네이션에 대한 평가로 볼 수있다.
    - **Answer relevancy(답변 적합성)**
        - 생성된 답변과 사용자의 질문간의 관련성을 평가하는 지표
        - 생성된 답변이 사용자의 질문과 얼마나 관련성이 있는지를 평가하는 지표.
- **Retrieval**
    -  질문에 대해 검색한 문서(context)들에 대한 평가
    -  **Context Precision(문맥 정밀도)**
        -  검색된 문서(context)들 중 질문과 관련 있는 것들이 **얼마나 상위 순위에 위치하는지** 평가하는 지표.
    -  **Context Recall(문맥 재현률)**
        -  검색된 문서(context)가 정답(ground-truth)의 정보를 얼마나 포함하고 있는지 평가하는 지표.
- 이러한 지표들은 RAG 파이프라인의 성능을 다각도로 평가하는 데 활용된다.
![RAGAS_score2](figures/RAGAS_score2.png)

# GUIDE
- 위 그림에서 box안에 파란색 빨간색이 평가 대상으로 보면된다.
- 즉 generation은 Answer에 대한 평가지표이므로 => answer와 context: faithfulness, answer와 user input이 answer relevancy 이런식으로 설명한다.

## Precision@K 
- K 상위 K개 라고 하는데 이건 top-k 의 상위 k를 말한다.
- precision@3 이면 상위 3개중 찾은 문서가 몇개 포함되어있나
  - 정답: [1, 2, 3, 4] , 찾은 문서: [2, 1, 5, 6] - precision@3 은 상위 3개중 2와 1 두개가 포함되 있으므로 2/3 이된다.

> - claim 뜻: 주장, 요청

## 주요 평가지표
### Generation 평가
- LLM이 생성한 답변에 대한 평가
  
#### Faithfulness (신뢰성)
- 생성된 답변이 얼마나 주어진 검색 문서들(context)를 잘 반영해 생성되었는지 평가. 할루시네이션에 대한 평가라고 할 수 있다. 
- 점수범위: **0 ~ 1** (1에 가까울수록 좋음)
- 답변에 포함된 모든 주장이 context에서 얼마나 추출 가능한지를 확인.

##### 평가 방법
1. Answer에서 주장 구문(claim statement)들을 생성(추출)한다. (주장이란, 질문(user input)과 관련된 내용)
    - 예) 
        - **질문**: 한국의 수도는 어디이고 인구는 얼마나 되나요? 
        - **LLM 답변**: 한국의 수도는 서울이고 인구수는 3000만명이다. 
        - **주장(claim)**: 
            1. 한국의 수도는 서울이다.
            2. 인구수는 3000만명이다.
2. 각 주장들을 context로 부터 추론 가능한지 판단. 이를 바탕으로 faithfulness 점수를 계산.
    - 예)
        - context: 한국은 동아시아에 위치하고 있는 나라다. 한국의 수도는 서울이다. .... 한국의 인구는 5000만명이고 서울에 1000만이 살고 있다.
        - 위 context에서 추론 가능한 주장: 
            - 한국의 수도는 서울이다. -> context에서 추론가능한 주장.
            - 한국의 인구는 3000만명이다. -> context에서 추론 불가능한 주장.
3. **Faithfulness score** 를 계산. 총 주장 수 중에서 context로 부터 추론가능한 주장의 개수.    
    - 예)
        - Faithfulness Score = $\cfrac{1}{2} = 0.5$ (두 개의 주장 중 한 개의 주장만 context에서 유추할 수있다.)
    - LLM 답변에서 주장을 추출 하는 것과 각 주장이 context에서 추론 가능한 지를 판단하는 것은 LLM 을 활용.
- 공식
    $$
    \text{Faithfulness Score}\;=\;\cfrac{\text{주어진\;context\;에서\;추론할\;수\;있는\;주장의\;개수}}{\text{총\;주장\;개수}}
    $$

### Answer relevancy (답변 적합성)
- 생성된 답변이 질문(user input)에 얼마나 잘 부합하는 지를 평가.
- 점수 범위: -1~1 (1에 가까울수록 좋음)
- LLM이 생성한 답변을 기반으로 질문들을 생성한다. 이렇게 생성한 질문들과 실제 질문(user input) 간의 유사도를 측정.

#### 평가방법
1. LLM이 생성한 답변을 기반으로 질문들을 생성.
    - 예) 
        - **LLM** 답변: 한국의 수도는 서울이고 인구수는 3000만명이다. 
        - **생성된 질문**: 
            1. 한국의 수도는 어디이고 인구는 얼마나 되나요?
            2. 한국의 수도는 어디인가요?
            3. 한국의 인구는 얼마나 되나요?
2. 실제 질문과 생성한 질문간의 코사인 유사도를 측정. 그 평균이 최종 점수가 된다.
    - 예)
        - **실제 질문**: 한국의 수도는 어디이고 인구는 얼마나 되나요?
        - **생성된 질문**: 
            1. 한국의 수도는 어디이고 인구는 얼마나 되나요?
            2. 한국의 수도는 어디인가요?
            3. 한국의 인구는 얼마나 되나요?
- 공식
  $$
    \cfrac{1}{N} \sum_{i=1}^{N} \text{cosine\_similarity}(q_{\text{user}_{_i}}, q_{\text{generated}})
  $$

## Retrieval 평가
Vector store에서 검색한 context에 대한 평가

### Context Precision
- 검색된 문서(context)들 중 **질문과 관련 있는 것들이 얼마나 상위 순위**에 있는 지 평가.
- 점수 범위: 0~1 (1에 가까울수록 좋음)


#### 평가방법

- 공식
$$
 \text{Context\;Precision@K} = \frac{\sum_{k=1}^{K} \left( \text{Precision@k} \times v_k \right)}{\ 상위\;K개\;결과에서의\;관련\;항목\;수}
$$
$$
 \text{Precision@k} = \frac{\text{True\;positive@k}}{(\text{True\;positive@k} + \text{False\;positive@k})} \\
$$
- $\text{Precision@k}$: 개별 문서에 대한 Precision
- K: context 의 개수(chuck 수)
- $v_k$: 관련성 여부로 0 또는 1. (0: 관련 없음, 1: 관련 있음)

#### 예시
- 질문과 context 관련성 예
    - 질문: 한국의 수도는 어디이고 인구는 얼마나 되나요?
    - 높은 정밀도 context
        - 한국의 수도는 서울이고 인구는 5000명 입니다. 
        - 한국의 수도는 서울입니다.
        - 한국은 동아시아에 위치해 있는 국가로 수도는 서울입니다.
        - 한국의 인구는 5000만명 입니다.
    - 낮은 정밀도 context
        - 한국은 동아시아에 위치한 국가입니다.
        - 한국의 K-pop은 전 세계적으로 유명합니다.
        - 비빔밥, 불고기는 한국의 대표적인 음식입니다.
    - **높은 정밀도의 context이 상위 순위에 위치했으면 높은 점수를 받는다.**
- 상위 5개의 검색 결과 중 1번째, 3번째, 4번째 문서가 관련이 있다고 가정
```bash
Precision@1 = 1/1 = 1.0    # True positive@1/(True positive@1 + False positive@1).  
Precision@2 = 1/2 = 0.5     
Precision@3 = 2/3 ≈ 0.67    
Precision@4 = 3/4 = 0.75
Precision@5 = 3/5 = 0.6
```
- vk의 값
    - 1번째: $v_1 = 1$
    - 2번째: $v_2 = 0$
    - 3번째: $v_3 = 1$
    - 4번째: $v_4 = 1$
    - 5번째: $v_5 = 0$

- Context Precision@5
$$
\text{Context\;Precision@5} = \frac{(1.0 \times 1) + (0.5 \times 0) + (0.67 \times 1) + (0.75 \times 1) + (0.6 \times 0)}{3} = \frac{1.0 + 0 + 0.67 + 0.75 + 0}{3} ≈ 0.807
$$

# GUIDE
- 예를 들어, 상위 5개의 검색 결과 중 1번째, 3번째, 4번째 문서가 관련이 있다고 가정하면:
```bash
Precision@1 = 1/1 = 1.0     # 1등 -> 1/1  --> 1 등이 관련 없는 것이면 전체 평균 점수가 확 낮아 질 수있다.
Precision@2 = 1/2 = 0.5     # 2등 -> 1/2  (1, 2등 두개 중에 하나만 관련(1번)이 있다.)
Precision@3 = 2/3 ≈ 0.67    
Precision@4 = 3/4 = 0.75
Precision@5 = 3/5 = 0.
```
왜 분모가 1, 2, 3, 4, 5 -> 그 순위에는 상위순위와 자신밖에 없으므로 그 개수가 분모. (3등은 1, 2, 3 등 세개 문서로.)

### Context Recall (문맥 재현률)
- 검색된 문서(context)가 얼마나 정답(ground-truth)의 정보를 포함있는 지 평가하는 지표
- 점수 범위: 0~1 (1에 가까울수록 좋음)
- **정답(ground truth)의 각 주장(claim)이 검색된 context와 얼마나 일치**하는지 계산함.

#### 평가방법
1. 정답에서 주장(claim)들을 생성(추출)한다.
    - 예) 
        - **정답**: 한국의 수도는 서울이고 인구수는 5000만명이다. 
        - **주장(claim)**: 
            1. 한국의 수도는 서울이다.
            2. 인구수는 5000만명이다.
2. 각 주장(claim)의 정보를 검색된 contexts에서 찾을 수 있는지 판별. 이를 바탕으로 context recall 점수를 계산.
    - 예)
        - context: 한국은 동아시아에 위치하고 있는 나라다. 한국의 수도는 서울이다.
        - 위 context에서 추론 가능한 주장: 
            - 한국의 수도는 서울이다. -> context에서 찾을 수 있다.
            - 한국의 인구는 5000만명이다. -> context에서 찾을 수 없다.
3. **Context Recall Score** 를 계산한다. 총 주장 수 중에서 context로 부터 찾을 수 있는 주장의 개수.
    - 예)
        - Context Recall Score = $\cfrac{1}{2} = 0.5$ (두 개의 주장 중 한 개의 주장만 context에서 찾을 수 있다.)

- 공식
    $$
    \text{Context Recall Score}\;=\;\cfrac{\text{GT의\;주장\;중\;주어진\;context\;에서\;찾을\;수\;있는\;주장의\;개수}}{\text{GT의\;총\;주장\;개수}}
    $$ 

In [ ]:
################# 12/30(화) 9:30 ######################

# RAGAS 평가 실습

In [ ]:
# !uv pip install ragas rapidfuzz
# 설치 후 커널 재시작

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_openai import ChatOpenAI
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.models import Distance, SparseVectorParams, VectorParams
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
###############################################################
# 데이터 준비
##############################################################

def load_and_split_olympic_data(file_path="data/olympic_wiki.md"):
    with open(file_path, "r", encoding="utf-8") as fr:
        olympic_text = fr.read()

    # Split
    splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=[
            ("#", "Header 1"),
            ("##", "Header 2"),
            ("###", "Header 3"),
        ],
        # strip_headers=False, # 문서에 header 포함 여부(default: True - 제거)
    )

    return splitter.split_text(olympic_text)

In [5]:
#################################################################
# Vector DB 연결
# retriever 생성
#################################################################

def get_vectorstore(collection_name: str = "olympic_info_wiki"):


    dense_embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

    client = QdrantClient(host="localhost", port=6333)

    # 컬렉션 삭제
    if client.collection_exists(collection_name):
        result = client.delete_collection(collection_name=collection_name)

    # 컬렉션 생성
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=3072, distance=Distance.COSINE),
      
    )

    vectorstore = QdrantVectorStore(
        client=client,
        collection_name=collection_name,    
        embedding=dense_embeddings
    )
    
    ######################################
    # Document들 추가
    ######################################
    documents = load_and_split_olympic_data()
    vectorstore.add_documents(documents=documents)

    return vectorstore


def get_retriever(vectorstore, k: int = 5):
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": k}
    )
    return retriever

In [4]:
vectorstore = get_vectorstore()

retriever = get_retriever(vectorstore)
retriever

VectorStoreRetriever(tags=['QdrantVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_qdrant.qdrant.QdrantVectorStore object at 0x1353fa660>, search_kwargs={'k': 5})

In [ ]:
################################################################################
# 평가할 RAG Chain
################################################################################

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document


vectorstore = get_vectorstore()
retriever = get_retriever(vectorstore)

prompt_txt = """# Instruction:
당신은 정보제공을 목적으로하는 유능한 AI Assistant 입니다.
주어진 context의 내용을 기반으로 질문에 답변을 합니다.
Context에 질문에 대한 명확한 정보가 있는 경우 그것을 바탕으로 답변을 합니다.
Context에 질문에 대한 명확한 정보가 없는 경우 "정보가 부족해 답을 할 수없습니다." 라고 답합니다.
절대 추측이나 일반 상식을 바탕으로 답을 하거나 Context 없는 내용을 만들어서 답변해서는 안됩니다.

# Context:
{context}

# 질문:
{query}
"""
prompt = ChatPromptTemplate.from_template(
    template=prompt_txt
)


def format_docs(documents:list)->str:
    """
    VectorStore에 조회한 문서들에서 내용(page_content)만 추출해서 str으로 합쳐서 반환.
    VectorStore의 검색결과인 List[Document]를 받아서 Document들에서 page_content의 내용만 추출한다.
    
    Args:
        documents(list[Document]): [Document(..), Document(...), ..]}
    Returns:
        str: 각 문서의 내용을 "\n\n"으로 연결한 string
    """
    return "\n\n".join(doc.page_content for doc in documents)

model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser()

# RAG 평가를 위해서 "답변", "검색한 문서" 둘이 출력되도록 변경.
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter

def format_doc_list(docs_dict: dict) -> list:
    # dictionary[list[Document], query:str] -> list[str]    
    # 문서 내용만 추출해서 (Document.page_content)만 추출한 리스트
    return [doc.page_contecnt for doc in docs_dict['context']]

# dict | dict -> dict
# RunnablePassthrough() | dict | dict ==> RunnableSequence
chain = RunnablePassthrough() | {
    "context":retriever,
    "query":RunnablePassthrough()
} | {
    "response": RunnableLambda(lambda x : x['context']) | format_docs | prompt | model | parser,
    "retrieved_context": format_doc_list, # RAGAS 평가 시 context -> List[str]
}

In [6]:
chain.invoke("1회 올림픽은 언제 어디서 열렸지")

'1896년 그리스 아테네에서 열렸습니다.'

# RAGAS 를 이용해 평가를 위한 합성 데이터 셋 만들기

- 평가 데이터셋 구성
  - `user_input`: 사용자 질문
  - `retrieved_contexts`: Vectorstore에서 검색한 context
  - `response`: LLM의 응답
  - `reference`: 정답

## TestsetGenerator
- **문서(retrieved_contexts)를 기준**으로 **질문**, **정답** 을 생성.
- 평가할 LLM으로 생성된 질문을 넣어 답변을 추출하여 데이터셋을 구성.

In [2]:
# 주피터노트북 환경에서 비동기적 처리 위해

import nest_asyncio
nest_asyncio.apply()

In [3]:
from qdrant_client import QdrantClient
########################################################
# 데이터셋을 생성할 때 사용할 context를 추출 - sampling
########################################################
client = QdrantClient(host="localhost", port=6333)
COLLECTION_NAME = "olympic_info_wiki"

# 전체 데이터를 다 조회해서 그 중 랜덤하게 K개만 sampling
info = client.get_collection(COLLECTION_NAME)
# info.points_count  # 저장된 데이터 개수
results, next_id = client.scroll(
    collection_name = COLLECTION_NAME,
    limit=info.points_count
)

# sampling
import random
sample_dataset = random.sample(results, 5)   # 리스트에서 랜덤하게 K개를 추출

# 문서 내용만 추출
docs = [point.payload['page_content'] for point in sample_dataset]
docs

['패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선수가 참가하는 국제 스포츠 대회로, 장애인 올림픽으로 불린다. 1948년에 루드비히 구트만 경(Sir Ludwig Guttman)은 제2차 세계대전에 참전한 군인들의 사회 복귀를 위한 일환으로 1948년 런던 올림픽과 동시에 몇몇 병원들을 연합해서 여러 경기를 펼쳤다. 구트만의 세계 휠체어, 신체부자유자대회(World Wheelchair and Amputee Games)로 알려진 이 대회는 매년 열리는 스포츠대회가 되었다. 12년이 넘도록 구트만과 다른 사람들은 스포츠를 상처를 치료하는 방법 중 하나로써 계속 대회 개최에 노력을 기울였다. 로마에서 열린 1960년 하계 올림픽때 구트만은 400명의 선수들을 "Parallel Olympics"에 참가시켰으며 이것이 곧 1회 패럴림픽으로 알려지게 되었다. 그 때부터 패럴림픽은 하계 올림픽이 열린 년도에 열리게 되었다. 서울에서 열린 1988년 하계 올림픽부터는 하계 올림픽을 개최한 도시는 패럴림픽도 같이 개최하기로 한다.',
 '올림픽에서 첫 번째 보이콧은 1956년 하계 올림픽에서 시작되었다. 네덜란드, 스페인, 스위스는 소련의 헝가리 침공에 항의해 참가를 거부했다. 캄보디아, 이집트, 이라크, 레바논은 제2차 중동 전쟁 때문에 보이콧했다. 1972년과 1976년 올림픽에는 많은 아프리카의 국가들이 남아프리카 공화국과 로디지아에서 일어나는 인종 차별정권에 대한 항의의 표시로 올림픽 참가를 거부했다. 이 보이콧에는 뉴질랜드도 관계가 되어있는데, 뉴질랜드 럭비 국가 대표팀이 당시 아파르트헤이트정책을 쓰던 남아프리카 공화국과 경기를 했음에도 불구하고 뉴질랜드의 올림픽 참가가 허용되었기 때문이었다. 국제 올림픽 위원회는 이 두 보이콧에 대해 심각하게 고민했으나 후자의 뉴질랜드의 경우는 럭비가 올림픽 종목이 아니라는 이유를 내세워 뉴질랜드의 올림픽 참가 금지 요청을 거부했다. 당시 아프리카에 속해 있던 20개국과 가이아나, 이라크는 경기를 끝낸 선수들이 있었

In [4]:
from ragas.testset import TestsetGenerator
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# gpt-5 버전은 사용할 수 없다. (temperature 설정 문제가 있다.)
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-large"))

generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
    llm_context="데이터셋은 반드시 한국어로 작성한다. 데이터셋은 JSON 문법을 꼭 지켜서 작성한다. 특히 구두점은 꼭 지켜야 한다. Document에 JSON 문법에 맞지 않는 표현이 있으면 반드시 수정해서 처리한다."
    # 질문/답변 생성할 때 llm에 전달할 추가 system prompt를 설정.
)

/var/folders/c1/rj7tb8cj0031l588c7fyvw0w0000gn/T/ipykernel_20791/2409063007.py:7: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
/var/folders/c1/rj7tb8cj0031l588c7fyvw0w0000gn/T/ipykernel_20791/2409063007.py:8: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-large"))


In [5]:
testset = generator.generate_with_chunks(
    docs, testset_size=10     # context 내용, 테스트 데이터셋 몇 개를 만들지.
)

Applying SummaryExtractor:   0%|          | 0/5 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/5 [00:00<?, ?it/s]

Node 615a4054-da44-46cc-b83d-a92a0354ce43 does not have a summary. Skipping filtering.
Node 71f03d99-8a8e-4bac-8e91-4a1156fbbba4 does not have a summary. Skipping filtering.
Node ff61046e-3df9-4b7d-8ccb-535a907a220c does not have a summary. Skipping filtering.
Node d3bef60a-a607-4727-bc53-cbb40463d852 does not have a summary. Skipping filtering.
Node 13006015-e33d-4e6c-8bfe-831346887ca1 does not have a summary. Skipping filtering.


Applying EmbeddingExtractor:   0%|          | 0/5 [00:00<?, ?it/s]

Applying ThemesExtractor:   0%|          | 0/5 [00:00<?, ?it/s]

...

Applying OverlapScoreBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Skipping multi_hop_abstract_query_synthesizer due to unexpected error: No relationships match the provided condition. Cannot form clusters.


Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

In [6]:
print(type(testset))
testset

<class 'ragas.testset.synthesizers.testset_schema.Testset'>


Testset(samples=[TestsetSample(eval_sample=SingleTurnSample(user_input='Sir Ludwig Guttman은 패럴림픽의 창설에 어떤 역할을 했습니까?', retrieved_contexts=None, reference_contexts=['패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선수가 참가하는 국제 스포츠 대회로, 장애인 올림픽으로 불린다. 1948년에 루드비히 구트만 경(Sir Ludwig Guttman)은 제2차 세계대전에 참전한 군인들의 사회 복귀를 위한 일환으로 1948년 런던 올림픽과 동시에 몇몇 병원들을 연합해서 여러 경기를 펼쳤다. 구트만의 세계 휠체어, 신체부자유자대회(World Wheelchair and Amputee Games)로 알려진 이 대회는 매년 열리는 스포츠대회가 되었다. 12년이 넘도록 구트만과 다른 사람들은 스포츠를 상처를 치료하는 방법 중 하나로써 계속 대회 개최에 노력을 기울였다. 로마에서 열린 1960년 하계 올림픽때 구트만은 400명의 선수들을 "Parallel Olympics"에 참가시켰으며 이것이 곧 1회 패럴림픽으로 알려지게 되었다. 그 때부터 패럴림픽은 하계 올림픽이 열린 년도에 열리게 되었다. 서울에서 열린 1988년 하계 올림픽부터는 하계 올림픽을 개최한 도시는 패럴림픽도 같이 개최하기로 한다.'], retrieved_context_ids=None, reference_context_ids=None, response=None, multi_responses=None, reference='Sir Ludwig Guttman은 1948년 제2차 세계대전에 참전한 군인들의 사회 복귀를 돕기 위해 런던 올림픽과 동시에 여러 병원들을 연합하여 경기를 개최하였고, 이 대회는 세계 휠체어, 신체부자유자대회로 알려지게 되었습니다. 이후 구트만은 1960년 로마 하계 올림픽에서 400명의 선수들을 "Parallel Olympics"에 참가시켰으며, 이것이 1회 패

In [ ]:
sample1 = testset.samples[0].eval_sample
print(sample1.user_input)              # 사용자 질문
print(sample1.reference_contexts)      # Vector DB의 context (이것을 바탕으로 user_input과 reference를 생성)
print(sample1.retrieved_contexts)      # RAG pipeline이 vector DB에서 검색한 문서
print(sample1.reference)               # 정답 (ground truth)
print(sample1.response)                # LLM 응답 (정답 추정값)

# 평가 시 필요한 속성: user_input, retrieved_contexts, reference, response
#        retrieved_context, response -> 평가할 RAG Chain으로부터 가져와야한다. 

Sir Ludwig Guttman은 패럴림픽의 창설에 어떤 역할을 했습니까?
['패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선수가 참가하는 국제 스포츠 대회로, 장애인 올림픽으로 불린다. 1948년에 루드비히 구트만 경(Sir Ludwig Guttman)은 제2차 세계대전에 참전한 군인들의 사회 복귀를 위한 일환으로 1948년 런던 올림픽과 동시에 몇몇 병원들을 연합해서 여러 경기를 펼쳤다. 구트만의 세계 휠체어, 신체부자유자대회(World Wheelchair and Amputee Games)로 알려진 이 대회는 매년 열리는 스포츠대회가 되었다. 12년이 넘도록 구트만과 다른 사람들은 스포츠를 상처를 치료하는 방법 중 하나로써 계속 대회 개최에 노력을 기울였다. 로마에서 열린 1960년 하계 올림픽때 구트만은 400명의 선수들을 "Parallel Olympics"에 참가시켰으며 이것이 곧 1회 패럴림픽으로 알려지게 되었다. 그 때부터 패럴림픽은 하계 올림픽이 열린 년도에 열리게 되었다. 서울에서 열린 1988년 하계 올림픽부터는 하계 올림픽을 개최한 도시는 패럴림픽도 같이 개최하기로 한다.']
None
Sir Ludwig Guttman은 1948년 제2차 세계대전에 참전한 군인들의 사회 복귀를 돕기 위해 런던 올림픽과 동시에 여러 병원들을 연합하여 경기를 개최하였고, 이 대회는 세계 휠체어, 신체부자유자대회로 알려지게 되었습니다. 이후 구트만은 1960년 로마 하계 올림픽에서 400명의 선수들을 "Parallel Olympics"에 참가시켰으며, 이것이 1회 패럴림픽으로 알려지게 되었습니다.
None


In [8]:
# Testset을 pandas DataFrame으로 변환 -> to_pandas(), to_xxxx()를 통해서 다른 구조로 변환이 가능.
eval_df = testset.to_pandas()
eval_df

,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name
0,Sir Ludwig Guttman은 패럴림픽의 창설에 어떤 역할을 했습니까?,[패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선수가 참가하는 국제 ...,Sir Ludwig Guttman은 1948년 제2차 세계대전에 참전한 군인들의 사...,Olympic Historian,PERFECT_GRAMMAR,MEDIUM,single_hop_specific_query_synthesizer
1,헝가리 올림픽 보이콧 역사에 대해 알려주세요. 헝가리와 관련된 올림픽 보이콧 사건이...,"[올림픽에서 첫 번째 보이콧은 1956년 하계 올림픽에서 시작되었다. 네덜란드, 스...","1956년 하계 올림픽에서 네덜란드, 스페인, 스위스는 소련의 헝가리 침공에 항의해...",Olympic Historian,MISSPELLED,LONG,single_hop_specific_query_synthesizer
2,베이징 올림픽 뭐 유명해요?,"[올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)...","2008 베이징 올림픽의 모든 종목 누적 시청자 수만 47억 명에 달하며, 이는 인...",Olympic Historian,POOR_GRAMMAR,SHORT,single_hop_specific_query_synthesizer
3,스페인 올림픽 위원회 관련해서 사마란치가 프랑코 정권에 협력했다는 비판이 있었나요?,[국제 올림픽 위원회(이하 IOC로 지칭)는 몇몇 위원들이 한 행위에 대해서 비판을...,사마란치 위원장 시기 때는 족벌 정치와 부패로 비난받았다. 사마란치가 스페인에서 프...,Olympic Historian,MISSPELLED,LONG,single_hop_specific_query_synthesizer
4,올림픽 역사에서 헬싱키가 갖는 정치적 의미와 소련의 첫 참가가 올림픽에 미친 영향에...,[쿠베르탱이 말했던 원래 이념과는 반대로 올림픽이 정치 혹은 체제 선전의 장으로 이...,소련은 헬싱키에서 열린 1952년 하계 올림픽 때 처음으로 참가했습니다. 그 전에는...,Olympic Historian,PERFECT_GRAMMAR,LONG,single_hop_specific_query_synthesizer
5,1956년 하계 올림픽에서 시작된 보이콧이 이후 다른 국가들의 올림픽 불참에 어떤 ...,[<1-hop>\n\n올림픽에서 첫 번째 보이콧은 1956년 하계 올림픽에서 시작되...,"1956년 하계 올림픽에서 네덜란드, 스페인, 스위스가 소련의 헝가리 침공에 항의해...",NaN,NaN,NaN,multi_hop_specific_query_synthesizer
6,"1968년 하계 올림픽에서 선수들이 정치적 성향 표현한 일 뭐였고, IOC는 어떻게...",[<1-hop>\n\n쿠베르탱이 말했던 원래 이념과는 반대로 올림픽이 정치 혹은 체...,1968년 하계 올림픽 육상 200m 경기에서 1위 토미 스미스와 3위 존 카를로스...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer
7,"1960년 하계 올림픽과 관련하여 패럴림픽이 어떻게 시작되었으며, 이 대회가 이후 ...",[<1-hop>\n\n패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선...,"패럴림픽은 신체·감각 장애가 있는 운동 선수를 위한 국제 스포츠 대회로, 1948년...",NaN,NaN,NaN,multi_hop_specific_query_synthesizer
8,"1960년 하계 올림픽 때 패럴림픽이 어떻게 시작됐고, 그 이후로 패럴림픽이 하계 ...",[<1-hop>\n\n패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선...,"패럴림픽은 신체·감각 장애가 있는 운동 선수가 참가하는 국제 스포츠 대회로, 194...",NaN,NaN,NaN,multi_hop_specific_query_synthesizer
9,"1960년 하계 올림픽과 관련하여 패럴림픽이 어떻게 시작되었으며, 이 대회가 이후 ...",[<1-hop>\n\n패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선...,"패럴림픽은 신체·감각 장애가 있는 운동 선수가 참가하는 국제 스포츠 대회로, 194...",NaN,NaN,NaN,multi_hop_specific_query_synthesizer


In [ ]:
chain.invoke(eval_df[0]['user_'])